In [5]:
# Third-party imports
import numpy as np
import pandas as pd
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)

load_dotenv()
excel_file = pd.ExcelFile('../data/raw_data_from_gsheet/RawData.xlsx')
all_sheet_names = excel_file.sheet_names
raw_data_sheets = [sheet for sheet in all_sheet_names if 'raw' in sheet.lower()]
daily_summaries_sheets = [sheet for sheet in all_sheet_names if 'daily' in sheet.lower()]


# Read in data files and merge like files
phase_change_data = pd.read_excel(excel_file, sheet_name='PhaseChanges')
phase_change_data.to_csv('../data/phase_change_data.csv', index=False)

# Raw data
raw_data_frames = []
for sheet_name in raw_data_sheets:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df['source_sheet'] = sheet_name.lower().replace(" raw data", "").replace(" ", "_").replace(",", "")
    raw_data_frames.append(df)
raw_data = pd.concat(raw_data_frames, ignore_index=True)
raw_data = raw_data[['Date', 'Bird', 'Time', 'LeftPecks', 'RightPecks', 'LeftSeed', 'RightSeed', 'source_sheet', 'Notes', 'CoolDown']]
raw_data.to_csv('../data/raw_data_all_locations.csv', index=False)

# Daily summaries
daily_summaries_frames = []
for sheet_name in daily_summaries_sheets:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df['source_sheet'] = sheet_name
    if "south" in sheet_name.lower():
        bird_columns = [col for col in df.columns if col not in ['Date', 'source_sheet', 'Feeder Visits']]
        df_melted = df.melt(id_vars=['Date', 'source_sheet'], 
                        value_vars=bird_columns, 
                        var_name='Bird', 
                        value_name='Visits')
        df_melted.rename(columns={'Visits': 'Feeder Visits'}, inplace=True)
        df_melted= df_melted[df_melted['Date'] != 'Grand Total'].reset_index(drop=True)
        df = df_melted.copy()
    daily_summaries_frames.append(df)
daily_sums = pd.concat(daily_summaries_frames, ignore_index=True)
daily_sums = daily_sums[['Date', 'source_sheet', 'Bird', 'Feeder Visits']]
daily_sums.to_csv('../data/daily_summaries_all_locations.csv', index=False)